In [1]:
# Note: use the latest version of Chromedriver

In [179]:
# Dependencies
from bs4 import BeautifulSoup
from datetime import datetime
import pymongo
import time
import requests
from splinter import Browser
import splinter #for exception handling
import os
import platform
import pandas as pd

In [182]:
def init_browser():
    # import platform
    if platform.system().lower() == 'windows'.lower():
        executable_path = {
            'executable_path': os.path.join(os.getcwd(), 'chromedriver.exe')}
    return Browser('chrome', **executable_path, headless=False)


def get_profile_urls(html):
    soup = BeautifulSoup(html, "html.parser")
    profile_urls = soup.find_all('a',class_="match-results-card" )
#     print(soup.prettify())
    return profile_urls


def scrape_profile_urls(browser,url):
    browser.reload()
    time.sleep(3)
    html = get_html(browser,url)
    profile_urls = get_profile_urls(html)
    return profile_urls

    
def login(browser):
    url_login ="https://www.okcupid.com/login"
    browser.visit(url_login)
    browser.fill('username', "eeggmm00@gmail.com")
    browser.fill('password', 'firebird8')
    #### Male profile (looking for women)
#     browser.fill('username', "coppertrial.2@gmail.com")
#     browser.fill('password', 'firebird8')
    
    
    button = browser.find_by_value('Sign in')
    button.click()
    time.sleep(5)
    return browser

def get_html(browser,url):   
    browser.visit(url)
    time.sleep(2)
    html = browser.html
    return html
    
def get_essays(html):
    soup = BeautifulSoup(html, "html.parser")
    essays = soup.body.find_all('p', class_="profile-essay-contents")
    return essays


def access_db(dbname,collectionname):
    
    conn = 'mongodb://localhost:27017'
    client = pymongo.MongoClient(conn)
    db = client[dbname]
    collection = db[collectionname]
    return db, collection

def get_gender(html):
    soup = BeautifulSoup(html, "html.parser")
    status = soup.body.find_all('div',class_="matchprofile-details-text")[0].text
    gender = status.split(', ')[1]
    return gender

def more_button():
    try:
        button = browser.find_by_text("More")
        if button[0]:
             button.click()
    except Exception as e:
        pass
#     except (splinter.exceptions.ElementDoesNotExist) as e:
#         print(e)    
    

def scrape_essays_and_gender(browser,url):
#     browser.reload()
    time.sleep(2)
    more_button()
    html = get_html(browser,url)
    essays = get_essays(html)
    gender = get_gender(html)
    return essays,gender


def essaystolist(essays):
    #function takes in a bunch of essays (tags that contain strings)
    #merges them together, and returns a list of each word
    essay_str = ""
    for essay in essays:
    #     print(essay.text)
        essay_str = essay_str + essay.text + " "


    essay_str = essay_str.replace("'s","’s").lower()

    
    puncmarks = [',','.','!','\\','?','/','"',"\n","'"]
    for puncmark in puncmarks:
        essay_str = essay_str.replace(puncmark," ")
#     print(essay_str)
    essaylist = essay_str.split(" ")
    essaylist[:] = (value for value in essaylist if value != '')
    return essaylist


def scrape_many(collection, browser, url_base):
    url = url_base + "/match"
    browser.visit(url)
    profile_urls = scrape_profile_urls(browser,url)
    print("size of profile_urls: ",len(profile_urls))
    
    num_refreshes = 3
    
    for num_refresh in range(num_refreshes):
        for profile_url in profile_urls[0:3]:
            try:
                url = url_base + profile_url['href']
                print(url)
                essays,gender = scrape_essays_and_gender(browser,url)
                #print(essays[0])
                #print(gender)
                #print("_______________")
                
                essayslist = essaystolist(essays)

                currenttime = datetime.now()
                collection.insert_many([
                            {'word':word, 'gender':gender, 
                             'datetime':currenttime, 'profile':browser.url
                            } for word in essayslist])
            
            except Exception as e:
                pass
                #print(e)
                #print("________________________")
                
        url = url_base + "/match"
        browser.visit(url)
        time.sleep(2)
        profile_urls = scrape_profile_urls(browser,url)
        print("This is the number of times we visited the match page:",num_refresh)
        
        
def main_login():
    browser = init_browser()
    browser = login(browser)
    return browser

def main(browser):
    

    db, collection = access_db('okcupid','frequentwords')
    collection.delete_many({}) ## Comment this line out if you don't want to delete the collection
    
#     browser = init_browser()
#     browser = login(browser)
    url_base = "https://www.okcupid.com"
    
    scrape_many(collection, browser,url_base)



browser = main_login()  #issues with login, comment out after logging in
main(browser)


# Data Analysis portion
data = pd.DataFrame(list(collection.find()))
data.groupby('word').count().sort_values(by='_id',ascending = False)


size of profile_urls:  20
https://www.okcupid.com/profile/4495611642637850280?cf=regular,matchsearch
https://www.okcupid.com/profile/18411647451244822504?cf=regular,matchsearch
https://www.okcupid.com/profile/5464183196718943287?cf=regular,matchsearch
This is the number of times we visited the match page: 0
https://www.okcupid.com/profile/6957809054954035383?cf=regular,matchsearch
https://www.okcupid.com/profile/4689099290095814189?cf=regular,matchsearch
https://www.okcupid.com/profile/12908650424097965682?cf=regular,matchsearch
This is the number of times we visited the match page: 1
https://www.okcupid.com/profile/12250287695654675290?cf=regular,matchsearch
https://www.okcupid.com/profile/12502692556782451296?cf=regular,matchsearch
https://www.okcupid.com/profile/3347872408875242422?cf=regular,matchsearch
This is the number of times we visited the match page: 2


,_id,datetime,gender,profile
word,,,,
i,25,25,25,25
and,23,23,23,23
a,22,22,22,22
to,20,20,20,20
the,17,17,17,17
of,12,12,12,12
just,6,6,6,6
with,6,6,6,6
be,6,6,6,6


In [152]:
pd.options.display.max_rows = 4000
table = data.groupby('word').count().sort_values(by='_id',ascending = False)
table

,_id,datetime,gender,profile
word,,,,
to,120,120,120,120
and,100,100,100,100
a,97,97,97,97
i,84,84,84,84
on,67,67,67,67
the,67,67,67,67
in,58,58,58,58
my,57,57,57,57
as,43,43,43,43
